# Homework 2: SQL in Python (Data and Databases 2018)

The next four homework assignments will be in this iPython notebook format. Read the instructions as you go, and type in the necessary code--then run the cells to see if you got it right. Don't worry if you are new to python for this assignment--the only "code" you will be writing are SQL statements that you will paste into each myquery variable.


## The Challenges

For this homework we will be using the tables we imported from the [UN data site](http://data.un.org/Explorer.aspx). You should already have `wind` and `solar` installed. As you will see, this homework asks that you find one more table of your choice from the UN's Energy statistics an import it into the the same database. [I have posted the instructions here](http://floatingmedia.com/columbia/csv_to_sql.html)

These are relatively simple CSV files--the only really important columns are `country`, `year` and `usage`. But the tricky part is that most countries have multiple entries: one for every year the UN has been gathering data on that country. So most of these queries are going to ask you to get electrical usage results based on country and year. (Major hint: JOINs are going to need to be based on country AND year.)

In the questions below, we focus on exploratory queries--with an emphasis on aggregates and joins. Read the questions carefully, and then write SQL statements that will get you the correct results. (You will want to explore tables and test out queries directly in the shell before you paste them into Python--the error handling for SQL in Python can be confusing, and there should be no errors in the prewritten Python parts of this document. Also, sometimes an error in Python will break your connection with the database--see conn.rollback() below for that solution.


First things first: Connect to database! 
(Make sure the cell is selected, then go to the cell menubar and choose "Run Cells")

In [4]:
import pg8000
conn = pg8000.connect(database="unenergy2018",user="postgres")
#conn = pg8000.connect(database="undata")
print(type(conn))

<class 'pg8000.core.Connection'>


If the connection isn't working make sure that you have typed in the name of the database correctly, and your username as well. 

If you get an error when running one of the SQL cells below, run this conn.rollback() cell to restart your connection.

In [5]:
conn.rollback()

## Question 1: Simple(ish) Query 

Create a statement that searches for the top 15 countries for solar energy usage in 2014. So figure out first what table you are searching and what columns you need (you can figure out the columns by looking at the sample results below). Then you need to test by year so you only get results from 2014, and order it correctly.

```
Germany 36056.0 mil
United States 24603.0 mil
Japan 24506.0 mil
Italy 22306.0 mil
China 15189.0 mil
Spain 13673.0 mil
France 5913.0 mil
India 5020.0 mil
Australia 4858.0 mil
United Kingdom 4040.0 mil
Greece 3792.0 mil
Belgium 2883.0 mil
Korea, Republic of 2557.0 mil
South Africa 2206.0 mil
Czechia 2123.0 mil
```

In [6]:
cursor = conn.cursor()
myquery = '''
SELECT country, usage FROM solar
WHERE year = 2014
ORDER BY usage DESC
NULLS LAST
LIMIT 15;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0], row[1], "mil")

Germany 36056.0 mil
United States 24603.0 mil
Japan 24506.0 mil
Italy 22306.0 mil
China 15189.0 mil
Spain 13673.0 mil
France 5913.0 mil
India 5020.0 mil
Australia 4858.0 mil
United Kingdom 4040.0 mil
Greece 3792.0 mil
Belgium 2883.0 mil
Korea, Republic of 2557.0 mil
South Africa 2206.0 mil
Czechia 2123.0 mil


## Question 2: aggregate query

Using avg(), get the average solar energy usage for each country, and it display only the countries with an average that is above 500 (million). (Remember for aggregates you need to use HAVING to test values, instead of WHERE). Make sure it is properly ordered like the results below, and wrap your avg column in round() to get round numbers:

```
19843.0 China
7297.0 Germany
5164.0 India
4514.0 United States
4042.0 Japan
3831.0 Italy
3030.0 Spain
1155.0 Greece
1147.0 Czechia
1134.0 France
1057.0 Belgium
902.0 United Kingdom
876.0 Australia
704.0 Bulgaria
547.0 Thailand
```

In [7]:
cursor = conn.cursor()
myquery = '''
SELECT round(avg(usage)), country FROM solar
GROUP BY country
HAVING avg(usage) > 500
ORDER BY avg(usage) DESC;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1])

19843.0 China
7297.0 Germany
5164.0 India
4514.0 United States
4042.0 Japan
3831.0 Italy
3030.0 Spain
1155.0 Greece
1147.0 Czechia
1134.0 France
1057.0 Belgium
902.0 United Kingdom
876.0 Australia
704.0 Bulgaria
547.0 Thailand


## Question 3: aggregate query 2

Get a list of countries whose Solar Energy data first started being collected in 2012. This is tricky to figure out, but the query should be pretty simple. The main thing you need to figure out is which aggregate function will find the earliest date, and then you need to test if year is 2012. And make it alphabetical by country.


```
2012 American Samoa
2012 Botswana
2012 Gabon
2012 Lithuania
2012 Marshall Islands
2012 Mauritius
2012 Poland
2012 South Sudan
2012 St. Helena and Depend.
2012 St. Kitts-Nevis
```


In [8]:
cursor = conn.cursor()
myquery = '''
SELECT min(year), country FROM solar
GROUP BY country
HAVING min(year) = 2012
ORDER BY country;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0], row[1])

2012 American Samoa
2012 Botswana
2012 Gabon
2012 Lithuania
2012 Marshall Islands
2012 Mauritius
2012 Poland
2012 South Sudan
2012 St. Helena and Depend.
2012 St. Kitts-Nevis


## Question 4: JOIN

Get the usage of solar AND wind power for Germany for each year in the tables. There should be four columns total, and you will be joining on two criteria.

(Please note that in the results below I am using Python to add in the strings "solar:" and "wind:")



```
Germany 1991 	wind: 215.0 	solar 1.0
Germany 1992 	wind: 291.0 	solar 4.0
Germany 1993 	wind: 674.0 	solar 3.0
Germany 1994 	wind: 1428.0 	solar 7.0
Germany 1995 	wind: 1712.0 	solar 7.0
Germany 1996 	wind: 2078.0 	solar 12.0
Germany 1997 	wind: 3034.0 	solar 18.0
Germany 1998 	wind: 4593.0 	solar 35.0
Germany 1999 	wind: 5528.0 	solar 30.0
Germany 2000 	wind: 9352.0 	solar 60.0
Germany 2001 	wind: 10456.0 	solar 116.0
Germany 2002 	wind: 15856.0 	solar 188.0
Germany 2003 	wind: 18713.0 	solar 313.0
Germany 2004 	wind: 25509.0 	solar 557.0
Germany 2005 	wind: 27229.0 	solar 1282.0
Germany 2006 	wind: 30710.0 	solar 2220.0
Germany 2007 	wind: 39713.0 	solar 3075.0
Germany 2008 	wind: 40574.0 	solar 4420.0
Germany 2009 	wind: 38647.0 	solar 6584.0
Germany 2010 	wind: 37793.0 	solar 11729.0
Germany 2011 	wind: 48883.0 	solar 19599.0
Germany 2012 	wind: 50670.0 	solar 26380.0
Germany 2013 	wind: 51708.0 	solar 31010.0
Germany 2014 	wind: 57357.0 	solar 36056.0
Germany 2015 	wind: 79206.0 	solar 38726.0
```

Hint: after the join you need to test that the country is Germany, then order by what?

In [9]:
cursor = conn.cursor()
myquery = '''
SELECT solar.country, solar.year, wind.usage, solar.usage
FROM solar JOIN wind ON solar.country = wind.country AND solar.year = wind.year
WHERE solar.country = 'Germany'
ORDER BY solar.year;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"\twind:",row[2],"\tsolar",row[3])

Germany 1991 	wind: 215.0 	solar 1.0
Germany 1992 	wind: 291.0 	solar 4.0
Germany 1993 	wind: 674.0 	solar 3.0
Germany 1994 	wind: 1428.0 	solar 7.0
Germany 1995 	wind: 1712.0 	solar 7.0
Germany 1996 	wind: 2078.0 	solar 12.0
Germany 1997 	wind: 3034.0 	solar 18.0
Germany 1998 	wind: 4593.0 	solar 35.0
Germany 1999 	wind: 5528.0 	solar 30.0
Germany 2000 	wind: 9352.0 	solar 60.0
Germany 2001 	wind: 10456.0 	solar 116.0
Germany 2002 	wind: 15856.0 	solar 188.0
Germany 2003 	wind: 18713.0 	solar 313.0
Germany 2004 	wind: 25509.0 	solar 557.0
Germany 2005 	wind: 27229.0 	solar 1282.0
Germany 2006 	wind: 30710.0 	solar 2220.0
Germany 2007 	wind: 39713.0 	solar 3075.0
Germany 2008 	wind: 40574.0 	solar 4420.0
Germany 2009 	wind: 38647.0 	solar 6584.0
Germany 2010 	wind: 37793.0 	solar 11729.0
Germany 2011 	wind: 48883.0 	solar 19599.0
Germany 2012 	wind: 50670.0 	solar 26380.0
Germany 2013 	wind: 51708.0 	solar 31010.0
Germany 2014 	wind: 57357.0 	solar 36056.0
Germany 2015 	wind: 79206.0 	

## Question 5: join with aggregates

Get the average wind usage and solar usage for each country. Order your results by average solar usage, and show only the top ten countries.

```
China wind: 161013.67 	solar: 19843.00
Germany wind: 24077.16 	solar: 7297.28
India wind: 30228.33 	solar: 5164.00
United States wind: 45975.35 	solar: 4514.35
Japan wind: 1684.00 	solar: 4042.46
Italy wind: 4050.92 	solar: 3830.77
Spain wind: 18894.12 	solar: 3029.77
Greece wind: 2751.42 	solar: 1155.08
Czechia wind: 338.60 	solar: 1147.20
France wind: 5171.45 	solar: 1133.50
```

In [10]:
cursor = conn.cursor()
myquery = '''


SELECT solar.country, avg(wind.usage), avg(solar.usage)
FROM solar JOIN wind ON solar.country = wind.country AND solar.year = wind.year
GROUP BY solar.country
ORDER BY avg(solar.usage) DESC
LIMIT 10;

'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],"wind:","%.2f" % row[1],"\tsolar:","%.2f" % row[2])

China wind: 161013.67 	solar: 19843.00
Germany wind: 24077.16 	solar: 7297.28
India wind: 30228.33 	solar: 5164.00
United States wind: 45975.35 	solar: 4514.35
Japan wind: 1684.00 	solar: 4042.46
Italy wind: 4050.92 	solar: 3830.77
Spain wind: 18894.12 	solar: 3029.77
Greece wind: 2751.42 	solar: 1155.08
Czechia wind: 338.60 	solar: 1147.20
France wind: 5171.45 	solar: 1133.50


## Question 6: same thing but slightly more complicated

Get the average wind usage and solar usage for each country, FROM 2010 TO THE PRESENT. Order your results by average solar usage, and show only the top ten countries.

```
China wind: 161013.67 	solar: 19843.00
Germany wind: 36398.50 	solar: 11394.69
United States wind: 72631.81 	solar: 6827.31
Japan wind: 2733.31 	solar: 6528.75
Italy wind: 6532.00 	solar: 6217.81
India wind: 30228.33 	solar: 5164.00
Spain wind: 30334.50 	solar: 4916.38
France wind: 7105.50 	solar: 1558.00
Australia wind: 3872.00 	solar: 1249.56
Greece wind: 2751.42 	solar: 1155.08
```

In [11]:
cursor = conn.cursor()
myquery = '''
SELECT solar.country, avg(wind.usage), avg(solar.usage)
FROM solar JOIN wind on solar.country = wind.country AND solar.year = wind.year
WHERE solar.year > 2009
GROUP BY solar.country
ORDER BY avg(solar.usage) DESC
LIMIT 10;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],"wind:","%.2f" % row[1],"\tsolar:","%.2f" % row[2])

Germany wind: 54269.50 	solar: 27250.00
China wind: 161013.67 	solar: 19843.00
Italy wind: 12884.67 	solar: 16400.17
United States wind: 150753.50 	solar: 16068.67
Japan wind: 4621.17 	solar: 15094.33
Spain wind: 48940.83 	solar: 11530.50
India wind: 30228.33 	solar: 5164.00
France wind: 15240.17 	solar: 4103.50
Australia wind: 7964.33 	solar: 3194.50
United Kingdom wind: 24402.33 	solar: 2541.00


## Question 7: similar, but even more challenging.

Get the average wind usage and solar usage for each country, FROM 2000 TO THE PRESENT. But only include countries that have solar and wind data from the year 2000. Hint: you will need to use a subquery (subtable) to pull out all of the countries that have data from 2000. And then you will need to join in both the solar and the wind table to that subquery. Don't put any limits on this query.

```
Germany wind: 24077.16 	solar: 7297.28
United States wind: 45975.35 	solar: 4514.35
Japan wind: 1684.00 	solar: 4042.46
Italy wind: 4050.92 	solar: 3830.77
Spain wind: 18894.12 	solar: 3029.77
France wind: 5171.45 	solar: 1133.50
Australia wind: 2818.77 	solar: 914.82
United Kingdom wind: 10732.82 	solar: 901.76
Korea, Republic of wind: 351.61 	solar: 515.57
Canada wind: 4917.46 	solar: 357.08
Switzerland wind: 30.45 	solar: 165.60
Austria wind: 1387.00 	solar: 148.00
Netherlands wind: 2363.36 	solar: 123.56
Portugal wind: 3416.27 	solar: 116.77
Denmark wind: 7548.65 	solar: 109.65
Martinique wind: 1.51 	solar: 40.86
Mexico wind: 1049.96 	solar: 33.62
Sri Lanka wind: 73.45 	solar: 12.39
Sweden wind: 2880.00 	solar: 10.96
Finland wind: 294.92 	solar: 3.12
```

In [13]:
cursor = conn.cursor()
myquery = '''
SELECT sub.country, avg(wind.usage), avg(solar.usage)
FROM solar 
JOIN wind ON solar.country = wind.country AND solar.year = wind.year
JOIN (SELECT solar.country, min(solar.year), min(wind.year)
FROM solar JOIN wind on solar.country = wind.country AND solar.year = wind.year
GROUP BY solar.country
HAVING min(solar.year) < 2001 AND min(wind.year) < 2001)
AS sub
ON wind.country = sub.country AND solar.country = sub.country
GROUP BY sub.country;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],"wind:","%.2f" % row[1],"\tsolar:","%.2f" % row[2])

Sweden wind: 2880.00 	solar: 10.96
Sri Lanka wind: 73.45 	solar: 12.39
Portugal wind: 3416.27 	solar: 116.77
Finland wind: 294.92 	solar: 3.12
France wind: 5171.45 	solar: 1133.50
United States wind: 45975.35 	solar: 4514.35
Netherlands wind: 2363.36 	solar: 123.56
Australia wind: 2818.77 	solar: 914.82
Spain wind: 18894.12 	solar: 3029.77
Italy wind: 4050.92 	solar: 3830.77
United Kingdom wind: 10732.82 	solar: 901.76
Germany wind: 24077.16 	solar: 7297.28
Canada wind: 4917.46 	solar: 357.08
Korea, Republic of wind: 351.61 	solar: 515.57
Japan wind: 1684.00 	solar: 4042.46
Denmark wind: 7548.65 	solar: 109.65
Switzerland wind: 30.45 	solar: 165.60
Austria wind: 1387.00 	solar: 148.00
Mexico wind: 1049.96 	solar: 33.62
Costa Rica wind: 399.35 	solar: 8.98


## Question 8: Building a basic timeseries

Step one: get a list of the first year of data and the most recent year of data for each country from the solar table. Limit the results to the first 15;

```
Costa Rica 1995 -- 2016
Cambodia 2005 -- 2015
Eritrea 1997 -- 2015
Guatemala 2014 -- 2015
Turkey 2014 -- 2015
Germany 1991 -- 2015
Madagascar 2008 -- 2015
Cyprus 2005 -- 2015
France 1994 -- 2015
Samoa 2013 -- 2015
Kiribati 2015 -- 2015
Slovenia 2007 -- 2015
St. Kitts-Nevis 2012 -- 2015
American Samoa 2012 -- 2015
Japan 1990 -- 2015
```

In [12]:
cursor = conn.cursor()
myquery = '''
SELECT country, min(year), max(year) FROM solar
GROUP BY country
LIMIT 15;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"--",row[2])

Indonesia 2010 -- 2015
Bangladesh 2013 -- 2015
Mayotte 2008 -- 2015
Brunei Darussalam 2011 -- 2015
Kiribati 2015 -- 2015
Luxembourg 2003 -- 2015
Sweden 1993 -- 2015
Jordan 2015 -- 2015
Dominican Republic 2011 -- 2016
Ireland 2011 -- 2015
Cambodia 2005 -- 2015
Sri Lanka 2000 -- 2015
American Samoa 2012 -- 2015
Portugal 1990 -- 2015
Finland 1991 -- 2015


## Question 9: Building a basic timeseries

Step two: Take that previous query, remove the limit, turn that into a subquery, and join the solar table on it twice, first to get the usage from the earliest date, next get the usage from the most recent date. Show the top 15 in usage from the most recent year.

Note: to join at the same table twice, you need to alias the table (solar as sl1, solar as sl2). This is getting challenging, so get as far as you can.
```
China 2013 -- 2015 5564.0 -- 38776.0
Germany 1991 -- 2015 1.0 -- 38726.0
Japan 1990 -- 2015 1.0 -- 35858.0
United States 1990 -- 2015 666.0 -- 35635.0
Italy 1990 -- 2015 4.0 -- 22942.0
Spain 1990 -- 2015 6.0 -- 13859.0
United Kingdom 1999 -- 2015 1.0 -- 7561.0
France 1994 -- 2015 1.0 -- 7259.0
India 2013 -- 2015 3433.0 -- 7039.0
Australia 1993 -- 2015 11.0 -- 5968.0
Greece 2004 -- 2015 1.0 -- 3900.0
Korea, Republic of 1990 -- 2015 1.0 -- 3880.0
Belgium 2004 -- 2015 1.0 -- 3065.0
Canada 1992 -- 2015 2.0 -- 2895.0
Thailand 2007 -- 2015 1.0 -- 2378.0
```

In [15]:
cursor = conn.cursor()
myquery = '''
SELECT sub.country, sub.small, sub.large, sl1.usage, sl2.usage
FROM (SELECT country, min(year) as small, max(year) as large FROM solar
GROUP BY country) AS sub JOIN solar AS sl1 ON sl1.country = sub.country
JOIN solar AS sl2 ON sl2.country = sub.country
WHERE sl1.year = sub.small AND sl2.year = sub.large
ORDER BY sl2.usage DESC
LIMIT 15;
'''  
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"--",row[2],row[3],"--",row[4])

China 2013 -- 2015 5564.0 -- 38776.0
Germany 1991 -- 2015 1.0 -- 38726.0
Japan 1990 -- 2015 1.0 -- 35858.0
United States 1990 -- 2015 666.0 -- 35635.0
Italy 1990 -- 2015 4.0 -- 22942.0
Spain 1990 -- 2015 6.0 -- 13859.0
United Kingdom 1999 -- 2015 1.0 -- 7561.0
France 1994 -- 2015 1.0 -- 7259.0
India 2013 -- 2015 3433.0 -- 7039.0
Australia 1993 -- 2015 11.0 -- 5968.0
Greece 2004 -- 2015 1.0 -- 3900.0
Korea, Republic of 1990 -- 2015 1.0 -- 3880.0
Belgium 2004 -- 2015 1.0 -- 3065.0
Canada 1992 -- 2015 2.0 -- 2895.0
Thailand 2007 -- 2015 1.0 -- 2378.0


## Question 10: join with aggregates, new data

Your final task: go back to the [UN data site](http://data.un.org/Explorer.aspx) and download another table of your choice from the energy statistics. [Full instructions here.](http://floatingmedia.com/columbia/csv_to_sql.html) Choose a table that has similar columns to the wind and energy tables, and input it into your database.  Make sure you test that everything is working in the shell before going back to Python.

Now, write a query that joins the third table to the query you just made. Get the average use for each Energy type: Solar, Wind, and the table you chose. Order it by the energy type you chose (that column should come first after the country). And display the top 10 countries by average Energy type, along with the averages for solar and wind.


In [14]:
cursor = conn.cursor()
myquery = '''
SELECT solar.country, avg(solar.usage), avg(wind.usage), avg(nuclear.usage)
FROM solar JOIN wind ON solar.country = wind.country AND solar.year = wind.year
JOIN nuclear ON solar.country = nuclear.country AND solar.year = nuclear.year
GROUP BY solar.country
ORDER BY avg(nuclear.usage) DESC
LIMIT 10;
'''
cursor.execute(myquery)
for row in cursor:
    print(row[0],"%.2f" % row[1],"%.2f" % row[2],"%.2f" % row[3])

United States 4514.35 45975.35 765234.19
France 1133.50 5171.45 420868.55
Japan 4042.46 1684.00 231300.23
Russian Federation 247.50 122.00 188113.50
Germany 7297.28 24077.16 146058.00
China 19843.00 161013.67 138313.33
Korea, Republic of 515.57 351.61 120824.39
Canada 357.08 4917.46 89651.71
Ukraine 319.43 605.57 87101.71
United Kingdom 901.76 10732.82 74984.06


Great work! You're done.